# HOMEWORK - 7 Link Analysis

## 7.1 Create a crawler using the webcrawler

### Create a Project

Go to the directory where you want to create the project

In [2]:
cd C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\

C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7


In [3]:
%%cmd
scrapy startproject culturetrip

Microsoft Windows [Version 10.0.17134.345]
(c) 2018 Microsoft Corporation. All rights reserved.

(py27) C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7>scrapy startproject culturetrip
New Scrapy project 'culturetrip', using template directory 'C:\\python\\envs\\py27\\lib\\site-packages\\scrapy\\templates\\project', created in:
    C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip

You can start your first spider with:
    cd culturetrip
    scrapy genspider example example.com

(py27) C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7>

Navigate to the folder which was just created.

In [4]:
cd culturetrip\

C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip


Create a spider using "scrapy genspider example example.com"

In [5]:
%%cmd
scrapy genspider culturetripspider https://theculturetrip.com

Microsoft Windows [Version 10.0.17134.345]
(c) 2018 Microsoft Corporation. All rights reserved.

(py27) C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip>scrapy genspider culturetripspider https://theculturetrip.com
Created spider 'culturetripspider' using template 'basic' in module:
  culturetrip.spiders.culturetripspider

(py27) C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip>

In [6]:
cd culturetrip\

C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip\culturetrip


In [7]:
cd spiders\

C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip\culturetrip\spiders


In [9]:
%%writefile culturetripcrawl.py
import scrapy
from scrapy.contrib.spiders import CrawlSpider, Rule
from scrapy.contrib.linkextractors.sgml import SgmlLinkExtractor

class culturetripitem(scrapy.Item):
    title = scrapy.Field()
    link = scrapy.Field()
    response = scrapy.Field()
    
class culturetripSpider(scrapy.Spider):
    name = 'culturetripcrawler'
    allowed_domains = ['https://theculturetrip.com/']
    start_urls = ['https://theculturetrip.com/north-america/usa/articles/the-usa-s-10-most-talented-food-bloggers-to-follow/',
                  'https://theculturetrip.com/north-america/film-and-tv/',
                  'https://theculturetrip.com/middle-east/',
                  'https://theculturetrip.com/asia/',
                  'https://theculturetrip.com/pacific/',
                  'https://theculturetrip.com/pacific/music/',
                  'https://theculturetrip.com/europe/books/',
                  'https://theculturetrip.com/north-america/']
    rules = (Rule(SgmlLinkExtractor(allow=(),restrict_xpaths=('//a[@class="button next"]',)),
                  callback="parse", 
                  follow= True),)



    def parse(self, response):
        hxs = scrapy.Selector(response)
        titles = hxs.xpath('//ul/li')
        item = []
        for title in titles:
            item_object = culturetripitem()
            item_object["title"] = title.xpath("a/text()").extract()
            item_object["link"] = title.xpath("a/@href").extract()
            item_object["response"] = response
            if item_object["title"] != []:
                item.append(item_object)

        return item

Overwriting culturetripcrawl.py


In [10]:
%%cmd
scrapy crawl culturetripcrawler -o ctcrawl.csv -t csv

Microsoft Windows [Version 10.0.17134.345]
(c) 2018 Microsoft Corporation. All rights reserved.

(py27) C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip\culturetrip\spiders>scrapy crawl culturetripcrawler -o ctcrawl.csv -t csv

(py27) C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip\culturetrip\spiders>

C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip\culturetrip\spiders\culturetripcrawl.py:2: ScrapyDeprecationWarning: Module `scrapy.contrib.spiders` is deprecated, use `scrapy.spiders` instead
  from scrapy.contrib.spiders import CrawlSpider, Rule
C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip\culturetrip\spiders\culturetripcrawl.py:3: ScrapyDeprecationWarning: Module `scrapy.contrib.linkextractors` is deprecated, use `scrapy.linkextractors` instead
  from scrapy.contrib.linkextractors.sgml import SgmlLinkExtractor
C:\Users\Akhila Sharon\Documents\GitHub\big-data-python-class\Homeworks\Homework-7\culturetrip\culturetrip\spiders\culturetripcrawl.py:3: ScrapyDeprecationWarning: Module `scrapy.contrib.linkextractors.sgml` is deprecated, use `scrapy.linkextractors.sgml` instead
  from scrapy.contrib.linkextractors.sgml import SgmlLinkExtractor
C:\Users\Akhila Sharon\Documents\GitHub\big-data-pyth

## 7.2  Create a Stochastic matrix from its resulting crawling

In [11]:
import pandas as pd
import numpy as np

mylist = []
raw = pd.read_csv("ctcrawl.csv")
raw['link'] = 'https://theculturetrip.com' + raw['link']
a = raw['response'][0]
mylist.append((a.split()[-1]).split('>')[-2])
for i in range(len(raw)):
    if raw['response'][i] == a:
        mylist.append(raw['link'][i])
    else:
        a = raw['response'][i]
        mylist.append((a.split()[-1]).split('>')[-2])
        
item = list(pd.DataFrame(mylist)[0].unique())

link = []
length = len(mylist)
for i, val in enumerate(mylist):
    if i < length-1:
        link.append((mylist[i], mylist[i+1]))

mydataframe = pd.DataFrame(index=item, columns=item)
df = mydataframe.replace(np.NaN, 0)
for i in link:
    df.loc[i] = 1.0

arr = np.array(df)
arr_sum = arr.sum(axis=1)

result = arr/arr_sum[:, np.newaxis]
matrix = np.nan_to_num(result)

matrix

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.]])

## 7.3 Pass it through the Page Rank algorithm and provide the list of the top 5 page URLs in your sample

### PageRank Algorithm

In [12]:
from numpy import *

def pagerank(H):
    n = len(H)
    w = zeros(n)
    rho = 1./n * ones(n);
    for i in range(n):
        if multiply.reduce(H[i]== zeros(n)):
            w[i] = 1
    newH = H + outer((1./n * w),ones(n))
 
    theta=0.85
    G = (theta * newH) + ((1-theta) * outer(1./n * ones(n), ones(n)))
    #print rho
    for j in range(10):
        rho = dot(rho,G)
        #print rho
    
    return list(rho)

#### Passing the matrix through page rank algorithm and listing the top 5 URL's

In [17]:
import pandas as pd
import numpy as np
mylist = []
raw = pd.read_csv("ctcrawl.csv")
raw['link'] = 'https://theculturetrip.com' + raw['link']
a = raw['response'][0]
mylist.append((a.split()[-1]).split('>')[-2])
for i in range(len(raw)):
    if raw['response'][i] == a:
        mylist.append(raw['link'][i])
    else:
        a = raw['response'][i]
        mylist.append((a.split()[-1]).split('>')[-2])
        
item = list(pd.DataFrame(mylist)[0].unique())

link = []
length = len(mylist)
for i, val in enumerate(mylist):
    if i < length-1:
        link.append((mylist[i], mylist[i+1]))

mydataframe = pd.DataFrame(index=item, columns=item)
df = mydataframe.replace(np.NaN, 0)
for i in link:
    df.loc[i] = 1.0

arr = np.array(df)
arr_sum = arr.sum(axis=1)

result = arr/arr_sum[:, np.newaxis]
matrix = np.nan_to_num(result)
pg = pagerank(matrix)
value = ((pd.DataFrame(pg)).sort_values(0, ascending=False)).head(5)
index = list(value.index)
url = pd.DataFrame(item)
print "Top five URLs:"
for i in index:
    print url[0][i]



Top five URLs:
https://theculturetrip.com/about-us/
https://theculturetrip.com/the-team/
https://theculturetrip.com/in-the-news/
https://theculturetrip.com/our-app/
https://theculturetrip.com/north-america/new-zealand/


##  7. 4 Extra Credit

###  Using Hits Algorithm

#### Hits Algorithm

In [19]:
def hits(A):
    n= len(A)
    Au= dot(transpose(A),A)
    Hu = dot(A,transpose(A))
    a = ones(n); h = ones(n)
    #print a,h
    for j in range(5):
        a = dot(a,Au)
        a= a/sum(a)
        h = dot(h,Hu)
        h = h/ sum(h)
    return h

#### Passing the matrix through page rank algorithm and listing the top 5 URL's

In [20]:
value = pd.DataFrame(hits(arr))
sort = (value.sort_values(0, ascending=False)).head(5)
hit = list(sort.index)
url = pd.DataFrame(item)
print "Top 5 page URLs:"
for i in hit:
    print  (url[0][i])

Top 5 page URLs:
https://theculturetrip.com/north-america/usa/articles/the-usa-s-10-most-talented-food-bloggers-to-follow/
https://theculturetrip.com/asia/cayman-islands/
https://theculturetrip.com/pacific/papua-new-guinea/
https://theculturetrip.com/north-america/papua-new-guinea/
https://theculturetrip.com/middle-east/kuwait/


# 7.5 Advanced Credit

## Weighted Page Rank



The weighted PageRank algorithm generalizes the simple PageRank algorithm in two ways:

1. Different pages can have different inherent qualities. In practice these can come from various measures of the page: how long it is, how well written it is on standard measures, how often it is accessed in query logs and so on.
2. Different outlinks on the same page can have different weights, according to various characteristics: Where they are in the page, how often they appear in the pages, emphasis by font, and so on.

Let us define:
For any page P, φ(P) is the inherent quality of P. 
Φ = ΣP φ(P). 
φ'(P) = φ(P)/Φ, the normalized quality of P. Thus ΣP φ'(P) = 1. 


For any pages P,Q, θ(P,Q) is the weight of the link from P to Q. 
Θ(P) = ΣQ θ(P,Q), the total weight of all the outlinks from P. 
θ'(P,Q) = θ(P,Q) / Θ(P), the normalized weight of the link P -> Q. Thus, ΣQ θ'(P,Q) = 1. 
If there are no outlinks from P, then define θ'(P,Q) = φ'(Q) for all pages Q.

Let f be a parameter between 0 and 1. We now posit that the important of page Q satisfies the following equation:

I(Q) = (1-f)*φ'(Q) + f*ΣP -> Q I(P) * θ'(P,Q)



### Implementing Weighted Page Rank Algorithm

code from https://gist.github.com/dheerajrajagopal/2915436

In [24]:
import numpy as np
from scipy.sparse import csc_matrix
import random

def column(matrix, i):
    return [row[i] for row in matrix]

def wpageRank(G, s = 0.85, maxerr = .001):
    """
    Computes the pagerank for each of the n states.
    Used in webpage ranking and text summarization using unweighted
    or weighted transitions respectively.
    Args
    ----------
    G: matrix representing state transitions
       Gij can be a boolean or non negative real number representing the
       transition weight from state i to j.
    Kwargs
    ----------
    s: probability of following a transition. 1-s probability of teleporting
       to another state. Defaults to 0.85
    maxerr: if the sum of pageranks between iterations is below this we will
            have converged. Defaults to 0.001
    """
    n = G.shape[0]

    # transform G into markov matrix M
    M = csc_matrix(G,dtype=np.float)
    rsums = np.array(M.sum(1))[:,0]
    ri, ci = M.nonzero()
    M.data /= rsums[ri]

    # bool array of sink states
    sink = rsums==0

    # Compute pagerank r until we converge
    ro, r = np.zeros(n), np.ones(n)
    while np.sum(np.abs(r-ro)) > maxerr:
        ro = r.copy()
        # calculate each pagerank at a time
        for i in xrange(0,n):
            # inlinks of state i
            Ii = np.array(M[:,i].todense())[:,0]
            # account for sink states
            Si = sink / float(n)
            # account for teleportation to state i
            Ti = np.ones(n) / float(n)
            # Weighted PageRank Equation
            r[i] = ro.dot( Ii*s + Si*s + Ti*(1-s)*G[i] ) 


    # return normalized pagerank
    return r/sum(r)

#### Passing the matrix through weighted page rank algorithm and listing the top 5 URL's

In [25]:
value = pd.DataFrame(wpageRank(arr))
sort = (value.sort_values(0, ascending=False)).head(5)
hit = list(sort.index)
url = pd.DataFrame(item)
print "Top 5 page URLs:"
for i in hit:
    print  (url[0][i])

Top 5 page URLs:
https://theculturetrip.com/advertise/
https://theculturetrip.com/get-in-touch/
https://theculturetrip.com/newsletter/
https://theculturetrip.com/north-america/papua-new-guinea/
https://theculturetrip.com/our-app/


## References
[1] https://doc.scrapy.org/en/latest/intro/tutorial.html
[2] https://github.com/mjhea0/Scrapy-Samples
[3] https://cs7083.wordpress.com/2013/01/31/demystifying-the-pagerank-and-hits-algorithms/
[4] https://gist.github.com/dheerajrajagopal/2915436
[5] https://cs.nyu.edu/courses/spring16/CSCI-GA.2580-001/prog3.html